# Modelos de Predicción
##### Futuras mejoras
* Estudiar bien los métodos de clasificación
* Cambiar el KNearestNeigh.. pues no está bien implementado: Falta normalizar, definir métrica, etc.
* Ordenar por tiempo y separar
* hacer crossvalidation
* 

## Importación de librerías

In [14]:
import os.path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

## Constantes

Rutas usuales que se ocuparán en el notebook

In [15]:
# Salvar gráficos
save_graf = False

# RUTAS
MAIN_PATH = os.path.join("..")

IMG_PATH = os.path.join(MAIN_PATH, "imagenes")
DATA_PATH = os.path.join(MAIN_PATH, "data")

WF_FOLDER_PATH = os.path.join(DATA_PATH,
                              "wildfires_us")
WF_DATA_PATH = os.path.join(WF_FOLDER_PATH,
                            "WILDFIRES_USA.csv")
WF_DATA_COLUMNS_PATH = os.path.join(WF_FOLDER_PATH,
                                    "WILDFIRES_USA_COLUMNS.csv")

# Tamaño de la imagen
my_figsize = (10, 5)

## Funciones auxiliares

Función que se ocupará para imprimir la información (número de filas y columnas) de un DataFrame.

In [17]:
def print_cantidad(dataframe):
    """Imprime la cantidad de datos que tiene el Data Frame.
    """
    msg_cantidad = "El dataset tiene una cantidad de {} datos y {} variables."
    print(msg_cantidad.format(dataframe.shape[0], dataframe.shape[1]))
    return None

## Carga de datos

### Columnas a ocupar

Se escojen las columnas a ocupar dependiendo de la importancia que tenga. Se omiten algunas columnas tales como las que son para el ID, como el nombre que tuvo el incendio, o la columna que indica de dónde se obtuvo el incendio; pues no deberían de afectar a la predicción.

In [18]:
# Todas las columnas
columnas = str(pd.read_csv(WF_DATA_COLUMNS_PATH).columns[0]).split(",")

# Columnas que se ocuparán en el análisis
columnas_ocupadas = columnas.copy()

# Columnas que no se ocuparán
columnas_sin_ocupar = [x for x in columnas if x not in columnas_ocupadas]

### Carga de Datos

In [19]:
df = pd.read_csv(WF_DATA_PATH)

# Convertimos los datos que sean fechas en ese tipo de dato
df["DISC_DATE_TIME"] = pd.to_datetime(df["DISC_DATE_TIME"])
df["CONT_DATE_TIME"] = pd.to_datetime(df["CONT_DATE_TIME"])

print_cantidad(df)

df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


El dataset tiene una cantidad de 581159 datos y 17 variables.


,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_NAME,DISC_DATE_TIME,DISC_MONTH,DISC_DAY_OF_WEEK,DISC_TIME,CONT_DATE_TIME,CONT_MONTH,CONT_DAY_OF_WEEK,CONT_TIME
0,2005,Miscellaneous,0.10,A,40.036944,-121.005833,CA,63,Plumas,2005-02-02 13:00:00,February,Wednesday,13,2005-02-02 17:30:00,February,Wednesday,17
1,2004,Lightning,0.25,A,38.933056,-120.404444,CA,61,Placer,2004-05-12 08:45:00,May,Wednesday,8,2004-05-12 15:30:00,May,Wednesday,15
2,2004,Debris Burning,0.10,A,38.984167,-120.735556,CA,17,El Dorado,2004-05-31 19:21:00,May,Monday,19,2004-05-31 20:24:00,May,Monday,20
3,2004,Lightning,0.10,A,38.559167,-119.913333,CA,3,Alpine,2004-06-28 16:00:00,June,Monday,16,2004-07-03 14:00:00,July,Saturday,14
4,2004,Lightning,0.10,A,38.559167,-119.933056,CA,3,Alpine,2004-06-28 16:00:00,June,Monday,16,2004-07-03 12:00:00,July,Saturday,12


## Preparación de la data

### Convertir variables categóricas a númericas

In [20]:
df_copy = df.copy()

drop_list = ["DISC_DATE_TIME", "CONT_DATE_TIME", "COUNTY", "FIPS_NAME", "FIRE_SIZE_CLASS"]

for to_drop in drop_list:
    df_copy = df_copy.drop(to_drop, axis=1)

columnas = [
    "STATE", 
    "DISC_MONTH",
    "DISC_DAY_OF_WEEK",
    "CONT_MONTH",
    "CONT_DAY_OF_WEEK",
#     "FIRE_SIZE_CLASS",
    "STAT_CAUSE_DESCR",
]

le = preprocessing.LabelEncoder()
for col in columnas:
    df_copy[col] = le.fit_transform(df_copy[col])

In [9]:
df_copy.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,DISC_MONTH,DISC_DAY_OF_WEEK,DISC_TIME,CONT_MONTH,CONT_DAY_OF_WEEK,CONT_TIME
0,2005,7,0.10,40.036944,-121.005833,4,3,6,13,3,6,17
1,2004,6,0.25,38.933056,-120.404444,4,8,6,8,8,6,15
2,2004,3,0.10,38.984167,-120.735556,4,8,1,19,8,1,20
3,2004,6,0.10,38.559167,-119.913333,4,6,1,16,5,2,14
4,2004,6,0.10,38.559167,-119.933056,4,6,1,16,5,2,12


### Preparación de la data de entrenamiento con la de prueba

In [21]:
from sklearn.model_selection import train_test_split

X = df_copy.drop("STAT_CAUSE_DESCR", axis=1).values
y = df.STAT_CAUSE_DESCR.values

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=0
                                                   )

## Generación de modelos

Empezamos a probar la clasificación con distintos métodos

### Naïve Bayes

In [22]:
from sklearn.naive_bayes import GaussianNB
# Entrenamos
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Predecimos
y_pred_gnb = gnb.predict(X_test)

# Mostramos
print(classification_report(y_test, y_pred_gnb))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                   precision    recall  f1-score   support

            Arson       0.35      0.17      0.23     27621
         Campfire       0.16      0.24      0.19      8562
         Children       0.02      0.08      0.03      3706
   Debris Burning       0.37      0.84      0.52     42022
    Equipment Use       0.17      0.01      0.03      9861
        Fireworks       0.07      0.09      0.08       788
        Lightning       0.61      0.45      0.52     34400
    Miscellaneous       0.42      0.10      0.17     30883
Missing/Undefined       0.16      0.02      0.03      8175
        Powerline       0.00      0.00      0.00      1662
         Railroad       0.00      0.00      0.00      1733
          Smoking       0.13      0.00      0.01      4502
        Structure       0.00      0.00      0.00       433

         accuracy                           0.35    174348
        macro avg       0.19      0.15      0.14    174348
     weighted avg       0.37      0.35      0.31    17

In [ ]:
gnb.class_prior_

### Clasificador de k-vecinos próximos

In [11]:
from sklearn.neighbors import KNeighborsClassifier
# Entrenamos
knc = KNeighborsClassifier(n_neighbors=5)
knc.fit(X_train, y_train)

# Predecimos
y_pred_knc = knc.predict(X_test)

# Mostramos
print(classification_report(y_test, y_pred_knc))

                   precision    recall  f1-score   support

            Arson       0.42      0.51      0.46     27621
         Campfire       0.32      0.31      0.31      8562
         Children       0.08      0.04      0.05      3706
   Debris Burning       0.51      0.61      0.56     42022
    Equipment Use       0.26      0.17      0.21      9861
        Fireworks       0.26      0.09      0.13       788
        Lightning       0.71      0.77      0.74     34400
    Miscellaneous       0.52      0.46      0.49     30883
Missing/Undefined       0.70      0.64      0.67      8175
        Powerline       0.19      0.04      0.07      1662
         Railroad       0.46      0.18      0.26      1733
          Smoking       0.12      0.02      0.03      4502
        Structure       0.03      0.00      0.00       433

         accuracy                           0.52    174348
        macro avg       0.35      0.30      0.31    174348
     weighted avg       0.50      0.52      0.51    17

### Análisis de Discriminante Lineal

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Entrenamos
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Predecimos
y_pred_lda = lda.predict(X_test)

# Mostramos
print(classification_report(y_test, y_pred_lda))

### Random Forest

In [31]:
from sklearn.ensemble import RandomForestClassifier
# Entrenamos
rfc = RandomForestClassifier(n_estimators=100, class_weight= "balanced", verbose=1, max_depth=5) # class_weight="balanced_subsample"
rfc = rfc.fit(X_train, y_train)

# Predecimos
y_pred_rfc = rfc.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   47.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.6s finished


In [32]:
# Mostramos
print(classification_report(y_test, y_pred_rfc))

                   precision    recall  f1-score   support

            Arson       0.52      0.33      0.40     27621
         Campfire       0.21      0.23      0.22      8562
         Children       0.09      0.11      0.10      3706
   Debris Burning       0.51      0.40      0.45     42022
    Equipment Use       0.23      0.06      0.10      9861
        Fireworks       0.05      0.42      0.10       788
        Lightning       0.61      0.72      0.66     34400
    Miscellaneous       0.58      0.42      0.49     30883
Missing/Undefined       0.36      0.72      0.48      8175
        Powerline       0.09      0.13      0.10      1662
         Railroad       0.04      0.19      0.06      1733
          Smoking       0.25      0.01      0.02      4502
        Structure       0.02      0.39      0.03       433

         accuracy                           0.42    174348
        macro avg       0.27      0.32      0.25    174348
     weighted avg       0.48      0.42      0.43    17

In [26]:
# Entrenamos
rfc = RandomForestClassifier(n_estimators=50) # class_weight="balanced_subsample"
rfc = rfc.fit(X_train, y_train)

# Predecimos
y_pred_rfc = rfc.predict(X_test)

In [27]:
# Mostramos
print(classification_report(y_test, y_pred_rfc))

                   precision    recall  f1-score   support

            Arson       0.56      0.55      0.56     27621
         Campfire       0.50      0.35      0.41      8562
         Children       0.22      0.03      0.06      3706
   Debris Burning       0.56      0.72      0.63     42022
    Equipment Use       0.39      0.22      0.28      9861
        Fireworks       0.49      0.14      0.21       788
        Lightning       0.76      0.86      0.81     34400
    Miscellaneous       0.55      0.58      0.56     30883
Missing/Undefined       0.82      0.76      0.79      8175
        Powerline       0.39      0.08      0.14      1662
         Railroad       0.79      0.27      0.40      1733
          Smoking       0.28      0.04      0.07      4502
        Structure       0.15      0.01      0.02       433

         accuracy                           0.60    174348
        macro avg       0.50      0.35      0.38    174348
     weighted avg       0.58      0.60      0.58    17